## Preamble

In [16]:
# noinspection PyUnresolvedReferences
import preamble

In [31]:
from os import getenv
from pathlib import Path

clinica_data_ci_dir = getenv("CLINICA_DATA_CI_DIR", str(Path.cwd().parent /  "clinica_data_ci"))

pipeline_dir = Path(clinica_data_ci_dir) / "data_ci" / "T1Linear"
pipeline_in_dir = pipeline_dir / "in"
pipeline_ref_dir = pipeline_dir / "ref"

## BIDS data grabber

Definition of the BDG interface.

In [18]:
from nipype.interfaces.io import BIDSDataGrabber

bids_data_grabber = BIDSDataGrabber(
    outfields=["T1w"],
    output_query={
        "T1w": {
            "datatype": "anat",
            "suffix": "T1w",
            "extension": [".nii.gz"],
        }
    }
)

## Bias field correction

Definition of the BFC interface.

In [19]:
from nipype.interfaces.ants import N4BiasFieldCorrection

n4_bias_field_correction = N4BiasFieldCorrection(bspline_fitting_distance=300)

## Registration

Definition of the registration interface.

In [20]:
from nipype.interfaces.ants import RegistrationSynQuick

registration_syn_quick = RegistrationSynQuick(transform_type="a")

## Cropping

Definition of the cropping interface.

In [21]:
from pydra.mark import annotate, task

@task
@annotate({"return": {"cropped_image": str}})
def crop_image(input_image: str, template_image: str) -> str:
    from pathlib import Path
    from nilearn.image import resample_to_img

    cropped_image = Path.cwd() / Path(input_image).name.replace(".nii.gz", "_cropped.nii.gz")

    resample_to_img(
        source_img=str(input_image),
        target_img=str(template_image),
        force_resample=True
    ).to_filename(cropped_image)

    return cropped_image

## Template fetchers

In [22]:
from pathlib import PurePath

def download_file(url: str, to: str) -> PurePath:
    from shutil import copyfileobj
    from ssl import SSLContext
    from urllib.request import urlopen

    print(f"Downloading {url} to {to}...")

    response = urlopen(url=url, context=SSLContext())
    with open(to, mode="wb") as f:
        copyfileobj(response, f)

    return PurePath(to)


@task
@annotate({"return": {"mni_template_file": PurePath}})
def download_mni_template() -> PurePath:
    from pathlib import Path

    return download_file(
        url="https://aramislab.paris.inria.fr/files/data/img_t1_linear/mni_icbm152_t1_tal_nlin_sym_09c.nii.gz",
        to=str(Path.cwd() / "mni_icbm152_t1_tal_nlin_sym_09c.nii.gz"),
    )


@task
@annotate({"return": {"ref_template_file": PurePath}})
def download_ref_template() -> PurePath:
    from pathlib import Path

    return download_file(
        url="https://aramislab.paris.inria.fr/files/data/img_t1_linear/ref_cropped_template.nii.gz",
        to=str(Path.cwd() / "ref_cropped_template.nii.gz"),
    )

## Workflow definition

In [35]:
from pydra import Workflow
from pydra.tasks.nipype1.utils import Nipype1Task

workflow = Workflow(
    name="t1_linear",
    input_spec=["input_dir"],
    input_dir=str(pipeline_in_dir / "bids"),
)

workflow.add(
    download_mni_template(name="download_mni_template")
)

workflow.add(
    download_ref_template(name="download_ref_template")
)

workflow.add(
    Nipype1Task(
        name="bids_data_grabber",
        interface=bids_data_grabber,
        base_dir=workflow.lzin.input_dir,
    )
)

workflow.add(
    Nipype1Task(
        name="n4_bias_field_correction",
        interface=n4_bias_field_correction,
        input_image=workflow.bids_data_grabber.lzout.T1w,
    )
)

workflow.n4_bias_field_correction.split("input_image")

workflow.add(
    Nipype1Task(
        name="registration_syn_quick",
        interface=registration_syn_quick,
        fixed_image=workflow.download_mni_template.lzout.mni_template_file,
        moving_image=workflow.n4_bias_field_correction.lzout.output_image,
    )
)

workflow.add(
    crop_image(
        name="crop_image",
        interface=crop_image,
        input_image=workflow.registration_syn_quick.lzout.warped_image,
        template_image=workflow.download_ref_template.lzout.ref_template_file,
    )
)

workflow.set_output([
    ("corrected_image", workflow.n4_bias_field_correction.lzout.output_image),
    ("warped_image", workflow.registration_syn_quick.lzout.warped_image),
    ("cropped_image", workflow.crop_image.lzout.cropped_image),
    ("xfm_file" , workflow.registration_syn_quick.lzout.out_matrix),
])

## Workflow execution

In [36]:
# Workflow execution
from pydra import Submitter

with Submitter(plugin="cf") as submitter:
    submitter(workflow)

results = workflow.result(return_inputs=True)

results

220504-16:54:39,944 nipype.workflow INFO:
	 [Node] Setting-up "bids_data_grabber" in "/private/var/folders/m3/r24ql8bj2h9_g1d970f9mr40000w73/T/tmprxn911vr/Nipype1Task_60c5cfc81966137047c39edcbf95662190397a0ddb24c89fb3ae60131721dccd/bids_data_grabber".
220504-16:54:39,949 nipype.workflow INFO:
	 [Node] Executing "bids_data_grabber" <nipype.interfaces.io.BIDSDataGrabber>
220504-16:54:40,359 nipype.workflow INFO:
	 [Node] Finished "bids_data_grabber", elapsed time 0.172443s.
220504-16:54:41,241 nipype.workflow INFO:
	 [Node] Setting-up "n4_bias_field_correction" in "/private/var/folders/m3/r24ql8bj2h9_g1d970f9mr40000w73/T/tmprxn911vr/Nipype1Task_639a4ada3ad475469b9c954c783d17c2a968b1a026dbc997a94c95122dd711f4/n4_bias_field_correction".
220504-16:54:41,244 nipype.workflow INFO:
	 [Node] Executing "n4_bias_field_correction" <nipype.interfaces.ants.segmentation.N4BiasFieldCorrection>
220504-16:56:04,423 nipype.workflow INFO:
	 [Node] Finished "n4_bias_field_correction", elapsed time 82.91216

({'t1_linear.input_dir': '/Users/ghislain.vaillant/Projects/clinica-pydra-testing/clinica_data_ci/data_ci/T1Linear/in/bids'},
 Result(output=Output(corrected_image=[PosixPath('/private/var/folders/m3/r24ql8bj2h9_g1d970f9mr40000w73/T/tmprxn911vr/Workflow_a8086032e0a2e2f1764d4f732e5fd70b4f80a66bc330cc7a389e1595f7109054/sub-ADNI022S0004_ses-M00_T1w_corrected.nii.gz')], warped_image=[PosixPath('/private/var/folders/m3/r24ql8bj2h9_g1d970f9mr40000w73/T/tmprxn911vr/Workflow_a8086032e0a2e2f1764d4f732e5fd70b4f80a66bc330cc7a389e1595f7109054/transformWarped.nii.gz')], cropped_image=[PosixPath('/private/var/folders/m3/r24ql8bj2h9_g1d970f9mr40000w73/T/tmprxn911vr/Workflow_a8086032e0a2e2f1764d4f732e5fd70b4f80a66bc330cc7a389e1595f7109054/transformWarped_cropped.nii.gz')], xfm_file=[PosixPath('/private/var/folders/m3/r24ql8bj2h9_g1d970f9mr40000w73/T/tmprxn911vr/Workflow_a8086032e0a2e2f1764d4f732e5fd70b4f80a66bc330cc7a389e1595f7109054/transform0GenericAffine.mat')]), runtime=None, errored=False))